In [ ]:
###check:
#  self.min_distance = 70 in CV_Pose_estimation2.py ? larger distance makes it easier

# class CVNavigation:
#     def __init__(self, control_inputs, width_size_video, height_size_video):
#         self.model = YOLO('yolo-Weights\yolov8x-worldv2.pt')

#use keyboar to control the players with:
# W A S D keys for player 1 and arrow keys for player 2 
# use T key to activate levels manually 


#cv_nav.set_detection_classes(['hat'])


#level =

In [ ]:
import cv2
import pygame
import sys
from CV_navigation import CVNavigation
from CV_pose_estimation import PoseEstimationRitual
import ultralytics
import random
import time
import json
ultralytics.checks()
pygame.init()
pygame.mixer.init()
# Constants
GRID_SIZE = 100
FPS = 30
WHITE = (255, 255, 255)
ORIGINAL_SCREEN_WIDTH = 1920
ORIGINAL_SCREEN_HEIGHT = 1080
width_size_video = 2304
height_size_video = 1344

language = "nl"  # This can be changed to "de", "es", "nl" as required



valid_color = (200, 200, 200, 128)  # Semi-transparent white for highlighting valid paths
green_color = (0, 255, 0, 128)  # Green with transparency
### Level 2 ###


### load sounds ###
start_fire_sound = pygame.mixer.Sound(r'sounds\lvl3_start_fire.mp3')
use_torch_sound = pygame.mixer.Sound(r'sounds\lvl3_torch.mp3')

stone_door_sound = pygame.mixer.Sound(r'sounds\lvl4_door.wav')
stone_door_sound.set_volume(2.0)
rune_magic_sound = pygame.mixer.Sound(r'sounds\lvl4_magic.mp3')
rune_magic_sound.set_volume(2.0)

# lvl5_magical_rune.mp3
magical_rune_sound = pygame.mixer.Sound(r'sounds\lvl5_magical_rune.mp3')
magical_rune_sound.set_volume(2.0)

#lvl6_startRitual.mp3
start_ritual_sound = pygame.mixer.Sound(r'sounds\lvl6_startRitual.mp3')
start_ritual_sound.set_volume(2.0)


#lvl6_teleport.wav
teleport_sound = pygame.mixer.Sound(r'sounds\lvl6_teleport.wav')
teleport_sound.set_volume(2.0)

#lvl7_EndRitual.wav
end_ritual_sound = pygame.mixer.Sound(r'sounds\lvl7_EndRitual.wav')
end_ritual_sound.set_volume(2.0)


help_sound = pygame.mixer.Sound(r'sounds\lvl2_help.aiff')
help_sound.set_volume(2.0)



# use_torch_sound.set_volume(2.0)
#use_torch_sound.play()

valid_path_level_x = {
    # all 18 columns in row 0:
    (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0),
    # all 18 columns in row 1:
    (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1),
    # all 18 columns in row 2:
    (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (15, 2), (16, 2), (17, 2), (18, 2),
    # all 18 columns in row 3:
    (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3), (7, 3), (8, 3), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3), (15, 3), (16, 3), (17, 3), (18, 3),
    # all 18 columns in row 4:
    (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4), (9, 4), (10, 4), (11, 4), (12, 4), (13, 4), (14, 4), (15, 4), (16, 4), (17, 4), (18, 4),
    # all 18 columns in row 5:
    (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (9, 5), (10, 5), (11, 5), (12, 5), (13, 5), (14, 5), (15, 5), (16, 5), (17, 5), (18, 5),
    # all 18 columns in row 6:
    (0, 6), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6), (6, 6), (7, 6), (8, 6), (9, 6), (10, 6), (11, 6), (12, 6), (13, 6), (14, 6), (15, 6), (16, 6), (17, 6), (18, 6),
    # all 18 columns in row 7:
    (0, 7), (1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7), (8, 7), (9, 7), (10, 7), (11, 7), (12, 7), (13, 7), (14, 7), (15, 7), (16, 7), (17, 7), (18, 7),
    # all 18 columns in row 8:
    (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8), (9, 8), (10, 8), (11, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8),
    # all 18 columns in row 9:
    (0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9), (9, 9), (10, 9), (11, 9), (12, 9), (13, 9), (14, 9), (15, 9), (16, 9), (17, 9), (18, 9)
}
speech_bubble_events = []  # List to manage active speech bubbles

### Initialize ###
def init_video():
    cap = cv2.VideoCapture(0)
    # Set resolution to  2304x1296 and feed the values to CVNavigation class
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width_size_video)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height_size_video)
    if not cap.isOpened():
        raise Exception("Could not open video device")
    cv2.namedWindow('Webcam Feed', cv2.WINDOW_NORMAL)
    return cap

# Function to load the appropriate language file
def load_language_file(language):
    file_path = f'dialogs_{language}.json'  # Load the language-specific JSON file
    with open(file_path, 'r') as file:
        return json.load(file)
def get_dialog(level, dialog_id):
    # 'language' is globally defined to indicate the current language
    return dialogs[level][dialog_id].get(language, "Text not available")

def load_resources(map_file):
    resources = {}
    resources['background'] = pygame.image.load(map_file).convert()
    resources['player_1_image'] = pygame.transform.scale(pygame.image.load('players\player_1.png').convert_alpha(), (GRID_SIZE, GRID_SIZE))
    resources['player_2_image'] = pygame.transform.scale(pygame.image.load('players\player_2.png').convert_alpha(), (GRID_SIZE, GRID_SIZE))
    resources['player_1_torch'] = pygame.transform.scale(pygame.image.load('players\player_1_torch.png').convert_alpha(), (GRID_SIZE, GRID_SIZE))
    resources['player_2_torch'] = pygame.transform.scale(pygame.image.load('players\player_2_torch.png').convert_alpha(), (GRID_SIZE, GRID_SIZE))
    
    return resources
def init_players():
    players = {}
    players['player1'] = {'position': [9, 5]}
    players['player2'] = {'position': [0,5]}
    return players
def map_init(map_file):
    pygame.display.set_caption("TreasureHunt")
    screen = pygame.display.set_mode((1920, 1024), pygame.RESIZABLE)
    screen = pygame.display.set_mode((pygame.display.Info().current_w, pygame.display.Info().current_h), pygame.RESIZABLE)
    resources = load_resources(map_file)
    return screen, resources
def draw_player(position, image, screen):
    screen.blit(image, (position[0] * GRID_SIZE, position[1] * GRID_SIZE))

def draw_grid_overlay(screen, grid_size, valid_path=None, green_path=None, red_path=None):
    screen_width, screen_height = screen.get_size()
    num_columns = screen_width // grid_size
    num_rows = screen_height // grid_size

    # Colors
    valid_color = (200, 200, 200)  # Light grey
    green_color = (0, 255, 0, 60)  # Green with transparency
    red_color = (255, 0, 0, 128)  # Red with transparency
    inactive_color = (50, 50, 50, 90)  # Dark grey for inactive areas

    # Create a special surface for transparency handling
    overlay = pygame.Surface((screen_width, screen_height), flags=pygame.SRCALPHA)

    # Draw the grid
    for x in range(num_columns):
        for y in range(num_rows):
            rect = pygame.Rect(x * grid_size, y * grid_size, grid_size, grid_size)
            if green_path and (x, y) in green_path:
                pygame.draw.rect(overlay, green_color, rect)
            elif red_path and (x, y) in red_path:
                pygame.draw.rect(overlay, red_color, rect)
            elif valid_path and (x, y) in valid_path:
                pygame.draw.rect(overlay, valid_color, rect, 1)  # Only draw border for valid path
            else:
                # Draw a semi-transparent fill for areas not included in the valid path if explicitly desired
                if not valid_path or (x, y) not in valid_path:
                    pygame.draw.rect(overlay, inactive_color, rect)

    screen.blit(overlay, (0, 0))


def exit_game():
    pygame.quit()
    sys.exit()


### Navigation functions ###
def update_player_position(event, players, screen_size):
    SCREEN_WIDTH, SCREEN_HEIGHT = screen_size
    # Player 1 arrow keys
    if event.key == pygame.K_LEFT:
        players['player1']['position'][0] = max(0, players['player1']['position'][0] - 1)
    elif event.key == pygame.K_RIGHT:
        players['player1']['position'][0] = min(SCREEN_WIDTH // GRID_SIZE - 1, players['player1']['position'][0] + 1)
    elif event.key == pygame.K_UP:
        players['player1']['position'][1] = max(0, players['player1']['position'][1] - 1)
    elif event.key == pygame.K_DOWN:
        players['player1']['position'][1] = min(SCREEN_HEIGHT // GRID_SIZE - 1, players['player1']['position'][1] + 1)
    
    # Player 2 W, A, S, D keys
    if event.key == pygame.K_a:
        players['player2']['position'][0] = max(0, players['player2']['position'][0] - 1)
    elif event.key == pygame.K_d:
        players['player2']['position'][0] = min(SCREEN_WIDTH // GRID_SIZE - 1, players['player2']['position'][0] + 1)
    elif event.key == pygame.K_w:
        players['player2']['position'][1] = max(0, players['player2']['position'][1] - 1)
    elif event.key == pygame.K_s:
        players['player2']['position'][1] = min(SCREEN_HEIGHT // GRID_SIZE - 1, players['player2']['position'][1] + 1)
def update_manual_controls(control_inputs, pressed_keys):
    """
    Update control inputs based on manually pressed keys.

    Parameters:
        control_inputs (dict): The dictionary tracking control inputs.
        pressed_keys (list): A list of keys that are currently pressed.
    """
    # Update the control inputs for the specified keys
    control_inputs['t'] = 't' in pressed_keys
    control_inputs['z'] = 'z' in pressed_keys
    control_inputs['u'] = 'u' in pressed_keys
    control_inputs['i'] = 'i' in pressed_keys
    control_inputs['o'] = 'o' in pressed_keys


def update_player_position_from_cv_level1(players, control_inputs, screen_size, valid_path):
    # Helper function to update a player's position
    def update_player(player_key, direction_keys, control_keys):
        player = players[player_key]
        current_x, current_y = player['position']

        # Calculate deltas based on control inputs
        dx = control_inputs.get(control_keys['right'], 0) - control_inputs.get(control_keys['left'], 0)
        dy = control_inputs.get(control_keys['down'], 0) - control_inputs.get(control_keys['up'], 0)

        new_x, new_y = current_x, current_y

        # Try moving in x direction
        if dx != 0:
            proposed_x = current_x + dx
            if (proposed_x, current_y) in valid_path:
                new_x = proposed_x

        # Try moving in y direction
        if dy != 0:
            proposed_y = current_y + dy
            if (new_x, proposed_y) in valid_path:
                new_y = proposed_y


        # Update position if changed
        if (new_x, new_y) != (current_x, current_y):
            
            player['position'] = [new_x, new_y]

    # Update player1 (using arrow keys)
    update_player(
        player_key='player1',
        direction_keys=['left', 'right', 'up', 'down'],
        control_keys={'left': 'left', 'right': 'right', 'up': 'up', 'down': 'down'}
    )

    # Update player2 (using WASD keys)
    update_player(
        player_key='player2',
        direction_keys=['a', 'd', 'w', 's'],
        control_keys={'left': 'a', 'right': 'd', 'up': 'w', 'down': 's'}
    )

    # Reset control inputs after processing, except for trigger keys (e.g., 't')
    for key in ['w', 'a', 's', 'd', 'up', 'down', 'left', 'right']:
        control_inputs[key] = 0

def show_speech_bubble(screen, speechbox_path, anchor_corner, bubble_position, text, font_path='freesansbold.ttf', font_size=20):
    # Load the speech bubble image
    speech_bubble = pygame.image.load(speechbox_path)
    bubble_width, bubble_height = speech_bubble.get_size()

    # Determine the position based on the anchor corner
    if anchor_corner == 'left':
        bubble_x, bubble_y = bubble_position[0], bubble_position[1] - bubble_height
    elif anchor_corner == 'right':
        bubble_x, bubble_y = bubble_position[0] - bubble_width, bubble_position[1] - bubble_height
    elif anchor_corner == 'bottom_right':
        bubble_x, bubble_y = bubble_position[0] - bubble_width, bubble_position[1]

    # Load the font and prepare the text
    font = pygame.font.Font(font_path, font_size)
    lines = text.split('\n')
    line_height = font.get_linesize()  # Get the height of each line
    total_text_height = line_height * len(lines)
    y_offset = (bubble_height - total_text_height) // 2  # Start vertically centered

    # Blit the speech bubble once before adding text
    screen.blit(speech_bubble, (bubble_x, bubble_y))

    for line in lines:
        text_surf = font.render(line, True, (0, 0, 0))  # Render text in black
        text_rect = text_surf.get_rect(center=(bubble_width // 2, y_offset + line_height // 2))
        screen.blit(text_surf, (bubble_x + text_rect.x, bubble_y + text_rect.y))
        y_offset += line_height  # Move to the next line

# ------------------- New Functions for Level 2 -------------------
def climb_together(players, valid_path):
    start_x = 10
    start_y = 5
    end_y = 2  # Players climb from y=5 up to y=2

    # Iterate through each step of the climb, starting from the top
    for y in range(start_y, end_y, -1):  # From y=5 down to y=2
        current_tile = (start_x, y)
        next_tile_up = (start_x, y - 1)

        # Check if both players are standing on the current tile
        if (players['player1']['position'] == [start_x, y] and players['player2']['position'] == [start_x, y]):
           

            # Unlock the next tile up
            valid_path.add(next_tile_up)

            # Only unlock the next tile once; break out of the loop
            break

# ------------------- New Functions for Level 4 -------------------
def create_circular_mask(radius, size):
    """Creates a simpler circular mask with a basic gradient effect."""
    mask = pygame.Surface(size, flags=pygame.SRCALPHA)
    # Center of the mask
    center = (size[0] // 2, size[1] // 2)
    # Create multiple layers of transparency for a basic gradient effect
    for i in range(10):  # Increase or decrease the range for a smoother or sharper gradient
        alpha = int(255 * (1 - (i / 10)))  # Decrease opacity as we move outwards
        pygame.draw.circle(mask, (255, 255, 255, alpha), center, radius - i * (radius // 10))
    return mask

def draw_light_effect(screen, grid_size, players_positions, map_image, dark_map_image, valid_path, green_path):
    num_columns, num_rows = map_image.get_width() // grid_size, map_image.get_height() // grid_size
    temp_surface = pygame.Surface(screen.get_size(), flags=pygame.SRCALPHA)
    temp_surface.blit(dark_map_image, (0, 0))  # Start by applying the dark map to the temp surface

    # Pre-compute the mask once (if all players have the same visibility radius)
    radius = int(grid_size * 2.5)
    mask_size = (grid_size * 5, grid_size * 5)  # Update this if you want a larger circular light mask
    mask = create_circular_mask(radius, mask_size)

    for px, py in players_positions:
        center_x, center_y = px * grid_size + grid_size // 2, py * grid_size + grid_size // 2
        
        # Change the range to (-2, 3) for a 5x5 grid centered on the player
        for dx in range(-2, 3):  # Adjusted for 5x5 grid
            for dy in range(-2, 3):  # Adjusted for 5x5 grid
                x, y = px + dx, py + dy
                if 0 <= x < num_columns and 0 <= y < num_rows:
                    rect = pygame.Rect(x * grid_size, y * grid_size, grid_size, grid_size)
                    temp_surface.blit(map_image, (x * grid_size, y * grid_size), rect)
                    
                    if (x, y) in valid_path:
                        pygame.draw.rect(temp_surface, valid_color, rect, 1)  # Highlight valid paths
                    if (x, y) in green_path:
                        pygame.draw.rect(temp_surface, green_color, rect)  # Fill cell with green color
        
        # Apply the precomputed mask to the temporary surface around the player
        mask_rect = pygame.Rect(center_x - radius, center_y - radius, *mask_size)
        temp_surface.blit(mask, mask_rect, special_flags=pygame.BLEND_RGBA_MIN)

    # Blit the temporary surface onto the main screen
    screen.blit(temp_surface, (0, 0))


def add_speech_bubble_event(screen, image, position, text, duration=3000):
    """Adds a speech bubble event with a duration in milliseconds."""
    event = {
        'image': image,
        'position': position,
        'text': text,
        'start_time': pygame.time.get_ticks(),
        'duration': duration
    }
    speech_bubble_events.append(event)

def show_active_speech_bubbles(screen):
    """Displays all active speech bubbles on the screen."""
    current_time = pygame.time.get_ticks()
    for event in speech_bubble_events[:]:
        if current_time - event['start_time'] <= event['duration']:
            # Show speech bubble if within duration
            show_speech_bubble(screen, event['image'], 'left', event['position'], event['text'])
        else:
            # Remove expired speech bubble
            speech_bubble_events.remove(event)

# ------------------- New Functions for Level 5 -------------------
# Function to calculate positions relative to the current screen size
def calculate_relative_position(position, screen_width, screen_height):
    x = int(position[0] / ORIGINAL_SCREEN_WIDTH * screen_width)
    y = int(position[1] / ORIGINAL_SCREEN_HEIGHT * screen_height)
    return (x, y)
# Visualize the rune sequence with blend-in, hold, blend-out effects
def visualize_sequence(screen, sequence, positions, image_map, players, resources):
    
    screen_width, screen_height = screen.get_size()
    adjusted_positions = positions  # Assuming positions are already in screen coordinates

    # Preload rune images outside of this function in your main code to avoid reloading
    
    try:
        rune_images = {
            'blue': pygame.transform.scale(pygame.image.load(r'maps\level_5\rune_blue.png').convert_alpha(), (GRID_SIZE, GRID_SIZE)),
            'green': pygame.transform.scale(pygame.image.load(r'maps\level_5\rune_green.png').convert_alpha(), (GRID_SIZE, GRID_SIZE)),
            'red': pygame.transform.scale(pygame.image.load(r'maps\level_5\rune_red.png').convert_alpha(), (GRID_SIZE, GRID_SIZE))
        }
        # load flaming table resources:

    except pygame.error as e:
        
        rune_images = { 'blue': None, 'green': None, 'red': None }
    
    # Time-related parameters
    blend_duration = 2000  # milliseconds for blend-in and blend-out
    hold_duration = 1500   # milliseconds to hold
    total_duration = blend_duration * 2 + hold_duration + 1000  # Total duration including pause

    start_time = pygame.time.get_ticks()
    clock = pygame.time.Clock()  # Create a Clock object for controlling frame rate
    speech_bubble_triggered = False
    magical_rune_sound.play()

    while pygame.time.get_ticks() - start_time < total_duration:
        elapsed_time = pygame.time.get_ticks() - start_time
        

        # Determine the alpha value for blending
        if elapsed_time < blend_duration:  # Blending in
            alpha = int((elapsed_time / blend_duration) * 255)
            
        elif blend_duration <= elapsed_time < blend_duration + hold_duration:  # Holding
            alpha = 255
            
        elif blend_duration + hold_duration <= elapsed_time < blend_duration * 2 + hold_duration:  # Blending out
            alpha = int(((blend_duration * 2 + hold_duration - elapsed_time) / blend_duration) * 255)
            
        else:
            alpha = 0  # After blending out
            

        # Handle events to keep the window responsive
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

        # Clear the screen
        screen.fill((0, 0, 0))
       

        # Blit the map image
        screen.blit(image_map, (0, 0))
        
        # Draw players
        draw_player(players['player1']['position'], resources['player_1_image'], screen)
        draw_player(players['player2']['position'], resources['player_2_image'], screen)
        

        # Draw runes with current alpha
        for rune_color, pos in zip(sequence, adjusted_positions):
            if rune_color is not None:  # If there is a rune to display
                rune_image = rune_images.get(rune_color)
                if rune_image:
                    # Create a copy to set alpha without altering the original
                    temp_image = rune_image.copy()
                    temp_image.set_alpha(alpha)
                    # Center the rune at the specified position
                    rune_rect = temp_image.get_rect(center=pos)
                    screen.blit(temp_image, rune_rect)
                    #print(f"Rune {rune_color} drawn at position {pos} with alpha {alpha}.")
                else:
                     print(f"xxxxx rune_color is None")
                     
            # else:
            #     print(f"No rune to display at position {pos}.")
        show_speech_bubble(screen, 'Speechbubbles\Speechbubble_1_left.png', 'left', (9 * GRID_SIZE, 4 * GRID_SIZE), get_dialog('level5', 'dialog_2'))
        # Update the display
        pygame.display.flip()

        # Control the frame rate to make visualization smooth
        clock.tick(30)  # Limit to 60 frames per second
    # After completing the visualization, pause for 1 second of nothing happening
    pygame.time.delay(500)
# Function to generate a random sequence of runes (blue, green, red, or None for blank)
def generate_sequence():
    import random
    rune_colors = ['blue', 'green', 'red', None]  # None represents a blank stone
    return random.sample(rune_colors, 3)

# ------------------- New Functions for Level 6 -------------------
pose_estimator = None

dialogs = load_language_file(language)


def game_loop():
    screen, resources = map_init('maps\level_1\map_level1.png')
    players = init_players()
    cap = init_video()  
    control_inputs = {
        'w': 0,
        'a': 0,
        's': 0,
        'd': 0,
        'up': 0,
        'down': 0,
        'left': 0,
        'right': 0,
        't': False,
        'z': False, 
        'u': False 
    }

    pressed_keys = []
    cv_nav = CVNavigation(control_inputs, width_size_video, height_size_video)
    
    level = 1 # Assuming Level 1 is where we use this logic

    clock = pygame.time.Clock()

    ### Declare variables for level 2 ###
    running = True
    visualize_mapchange = False
    player1_active = False
    level2_item_sharp_metal_part = False
    level2_climb_together = False
    rescue_bubble_time = None
    level_complete = False
    
    
        
    last_check_time_lvl5 = time.time()  # Set to the current time initially
    check_interval_lvl5 = 10  # Interval in seconds to check the sequence
    flame_display_time_lvl5 = 0  # Duration for showing the red or green flame
    current_sequence_index_lvl5 = 0  # Index for the current sequence (0, 1, or 2)

    pose_estimator=False

    default_font = pygame.font.SysFont('chiller', 36, bold=True)

    while running:
        ret, frame = cap.read()  # Capture frame here
        if not ret:
            print("Failed to capture frame")
            continue
        frame = cv2.flip(frame, 1)  # Flip the frame horizontally
       
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                pressed_keys.append(pygame.key.name(event.key))
                if event.key == pygame.K_ESCAPE:
                    running = False
                else:
                    update_player_position(event, players, screen.get_size())
        update_manual_controls(control_inputs, pressed_keys)

# ------------------- L E V E L 1 ------------------- #
        if level == 1:

            # Initialize variables for level 3
            if not 'level1_initialized' in locals():
                valid_path_level_1 = {
                (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), 
                (5, 6), (5, 7), (5, 8), 
                (6, 8), (7, 8), (8, 8), (9, 8), (10, 8), (11, 8), (12, 8), (13, 8),
                (13, 7), (13, 6), (13, 5), (13, 4), (13, 3),
                (12, 3), (11, 3), (10, 3), (9, 3), (8, 3),
                (8, 2), (8, 1), (8, 0)
                }   
                winning_position_level_1 = (8, 0)
                cv_nav.set_detection_classes(['hat'])

                level1_initialized = True

            screen.fill((255, 255, 255))
            screen.blit(resources['background'], (0, 0))
            
            draw_grid_overlay(screen, GRID_SIZE, valid_path_level_1, green_path={winning_position_level_1}, red_path=None)

            show_speech_bubble(screen, 'Speechbubbles\Speechbubble_1_left.png', 'left', (0 * GRID_SIZE, 5 * GRID_SIZE), get_dialog('level1', 'dialog_1'))
            draw_player(players['player2']['position'], resources['player_2_image'], screen)
            pygame.display.flip()
            update_player_position_from_cv_level1(players, control_inputs, screen.get_size(), valid_path_level_1)

            if players['player2']['position'] == list(winning_position_level_1):
                # darken the screen to indicate level change. count form 1 to 255 and back to 1:
                for i in range(1, 256):
                    screen.fill((i, i, i))
                    pygame.display.flip()
                level = 2 # Move to next level
# ------------------- L E V E L 2 ------------------- #
        if level == 2:
            screen.fill((255, 255, 255))
            screen.blit(resources['background'], (0, 0))
            

            # Initialize variables for level 3
            if not 'level2_initialized' in locals():
                level2_item_sharp_metal_part = False
                valid_path_level_2 = {
                    (6, 5), (17, 5),
                    (4, 6), (5, 6), (6, 6), (7, 6),(10,6), (15, 6), (16, 6), (17, 6),
                    (4, 7), (5, 7), (6, 7), (7, 7), (8, 7), (9, 7), (10, 7), (11, 7), (15,7),
                    (5, 8), (6, 8), (7, 8), (8, 8), (9, 8), (10, 8), (11, 8), (15, 8),
                    (6, 9), (7, 9), (8, 9), (9, 9), (10, 9), (11, 9), (12, 9), (13, 9), (14, 9), (15, 9)
                }
                green_path_lvl_2 = {(17, 5)}
                cv_nav.set_detection_classes(['hat'])  ### EDIT FOR WORKING KNIFE 
                cut_loose = False
                level2_initialized = True


            draw_grid_overlay(screen, GRID_SIZE, valid_path_level_2, green_path=green_path_lvl_2, red_path=None)
            update_player_position_from_cv_level1(players, control_inputs, screen.get_size(), valid_path_level_2)


            if not visualize_mapchange:
                for i in range(255, 0, -1):
                    screen.fill((i, i, i))
                    pygame.display.flip()
                screen, resources = map_init('maps\level_2\map_level2_1.png')  # Load the level 2 map
                players['player2']['position'] = [7, 9]
                visualize_mapchange = True
                speech_bubble_start_time = pygame.time.get_ticks()  
            # Show the speech bubble for 10 seconds
            if pygame.time.get_ticks() - speech_bubble_start_time <= 10000:
                show_speech_bubble(screen, 'Speechbubbles\Speechbubble_1_left.png', 'left', (9 * GRID_SIZE, 6 * GRID_SIZE),
                                    get_dialog('level2', 'dialog_1'))
            
            # Pickup Item: Sharp Metal Part
            if players['player2']['position'] == [17, 5] and not level2_item_sharp_metal_part: 
                green_path_lvl_2= {(9, 5)}
                cv_nav.set_detection_classes(['hat', 'yellow scissors'])
                valid_path_level_2.update({(11, 6), (10, 5), (9, 5)})
                draw_grid_overlay(screen, GRID_SIZE, valid_path_level_2, green_path=green_path_lvl_2)
                add_speech_bubble_event(screen, 'Speechbubbles\Speechbubble_1_left.png', (9 * GRID_SIZE, 5 * GRID_SIZE),
                                                get_dialog('level2', 'dialog_2'), 6000)
        
                level2_item_sharp_metal_part = True

            # use the sharp metal part 
            if players['player2']['position'] == [9, 5] and level2_item_sharp_metal_part and not player1_active:
                cv_nav.update_player_positions(players['player1']['position'], players['player2']['position'])
                cut_loose = cv_nav.get_cut_loose()
                

                if control_inputs.get('t', False) or cut_loose:
                    
                    rescue_bubble_time = pygame.time.get_ticks()  #
                    screen, resources = map_init('maps\level_2\map_level2_2.png')  
                    players['player1']['position'] = [9, 7] 
            
                    player1_active = True
                    # set control inputs to 0 after processing
                    
                    green_path_lvl_2.discard((9, 5))
                    green_path_lvl_2.add((10, 2))
                    valid_path_level_2.add((10, 2))
                    level2_climb_together = True
                    control_inputs['t'] = False

            if rescue_bubble_time and pygame.time.get_ticks() - rescue_bubble_time <= 7000:
                    show_speech_bubble(screen, 'Speechbubbles\Speechbubble_2_left.png', 'left', (9 * GRID_SIZE, 5 * GRID_SIZE),
                                    get_dialog('level2', 'dialog_3'))
                                    
            if level2_climb_together:
                climb_together(players, valid_path_level_2)
                
            if players['player1']['position'] == [10, 2] and players['player2']['position'] == [10, 2]:
                
                green_path_lvl_2.discard((10, 2))
                green_path_lvl_2.add((9, 0))
                valid_path_level_2.update({(9, 2), (9, 1), (9, 0)})


            ### Winning Condition for Level 2 ###
            if players['player1']['position'] == [9,0] and players['player2']['position'] == [9,0]:
                visualize_mapchange = False
                for i in range(1, 256):
                    screen.fill((i, i, i))
                    pygame.display.flip()
                
                level = 3
            

            draw_player(players['player2']['position'], resources['player_2_image'], screen)
            if player1_active:
                draw_player(players['player1']['position'], resources['player_1_image'], screen)
            
            show_active_speech_bubbles(screen)
            pygame.display.flip()  # Update the screen once all elements are drawn
# ------------------- L E V E L 3 ------------------- #
        if level == 3:
            
            screen.fill((255, 255, 255))
            screen.blit(resources['background'], (0, 0))


            # Initialize variables for level 3
            if not 'level3_initialized' in locals():
                
                valid_path_level_3 = {
                #row 5:
                (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (14, 5),
                #row 6:
                (0, 6), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6),              (14, 6), 
                # row 7:
                (0, 7), (1, 7), (2, 7), (3, 7), (4, 7),                     (14, 7), 
                # row 8:
                (0, 8), (1, 8), (2, 8), (3, 8),     (6, 8), (7, 8), (8, 8), (9, 8), (10, 8),        (14, 8), (15, 8), 
                # row 9:
                (0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9), (9, 9), (10, 9), (11, 9), (12, 9), (13, 9), (14, 9), (15, 9), (15,6), (15,7), (14,7)
                }
                green_path_lvl_3 = {(2, 7), (4, 7), (5, 5), (14,5)}
                                
                apple_tree_searched = False
                level3_found_lighter = False
                level3_fire_started = False
                level3_knows_about_cave = False
                
                player1_has_torch = False
                player2_has_torch = False
                cv_nav.set_detection_classes(['hat'])

                # Initialize speech bubble states and timers for each player
                player1_speech_bubble_active = False
                player1_speech_bubble_start_time = None
                player1_speech_bubble_duration = 10000  # Duration in milliseconds (10 seconds)

                player2_speech_bubble_active = False
                player2_speech_bubble_start_time = None
                player2_speech_bubble_duration = 10000  # Duration in milliseconds (10 seconds)

                # Initialize flags for players having a stick (assuming you have such flags)
                player1_has_stick = False
                player2_has_stick = False
                level3_initialized = True
            
            # Initialize Map level 3:
            if not visualize_mapchange:
                for i in range(255, 0, -1):
                    screen.fill((i, i, i))
                    pygame.display.flip()
                screen, resources = map_init('maps\level_3\map_level3.png')  
                players['player1']['position'] = [9, 9]
                players['player2']['position'] = [9, 9]
                visualize_mapchange = True
                speech_bubble_start_time = pygame.time.get_ticks()

            draw_grid_overlay(screen, GRID_SIZE, valid_path_level_3, green_path=green_path_lvl_3, red_path=None)
            update_player_position_from_cv_level1(players, control_inputs, screen.get_size(), valid_path_level_3)

            # Show initial speech bubble for 15 seconds:
            if pygame.time.get_ticks() - speech_bubble_start_time <= 8000:
                show_speech_bubble(screen, 'Speechbubbles\Speechbubble_1_left.png', 'left', (9 * GRID_SIZE, 7 * GRID_SIZE),
                                    get_dialog('level3', 'dialog_1'))
                # play help sound from \sounds\help.wav


            # If either player searches the camp (apple tree) /piucking up sticks:
            if players['player1']['position'] == [2, 7]:
                if level3_fire_started:
                    show_speech_bubble(screen, 'Speechbubbles\Speechbubble_2_left.png', 'left', (2 * GRID_SIZE, 7 * GRID_SIZE),
                                        get_dialog('level3', 'dialog_2'))
                    player1_has_stick = True
                else:
                    show_speech_bubble(screen, 'Speechbubbles\Speechbubble_2_left.png', 'left', (2 * GRID_SIZE, 7 * GRID_SIZE),
                                        get_dialog('level3', 'dialog_3'))
                    
            if players['player2']['position'] == [2, 7]:
                if level3_fire_started:
                    show_speech_bubble(screen, 'Speechbubbles\Speechbubble_2_left.png', 'left', (2 * GRID_SIZE, 7 * GRID_SIZE),
                                        get_dialog('level3', 'dialog_2'))
                    player2_has_stick = True
                else:
                    show_speech_bubble(screen, 'Speechbubbles\Speechbubble_2_left.png', 'left', (2 * GRID_SIZE, 7 * GRID_SIZE),
                                        get_dialog('level3', 'dialog_3'))
                    
            # If either player discovers the dark cave:
            if players['player1']['position'] == [14, 5] or players['player2']['position'] == [14, 5]:
                if player1_has_torch and player2_has_torch:
                    print("Both players have torches")
                else:
                    show_speech_bubble(screen, 'Speechbubbles\Speechbubble_2_left.png', 'left', (13 * GRID_SIZE, 5 * GRID_SIZE),
                                         get_dialog('level3', 'dialog_4'))
                    level3_knows_about_cave = True
                  
            # If either player searches the tent:
            if players['player1']['position'] == [5, 5] or players['player2']['position'] == [5, 5]:
                if level3_knows_about_cave:
                    show_speech_bubble(screen, 'Speechbubbles\Speechbubble_2_left.png', 'left', (5 * GRID_SIZE, 5 * GRID_SIZE),
                                         get_dialog('level3', 'dialog_5'))
                    cv_nav.set_detection_classes(['hat', 'blue lighter'])
                    level3_found_lighter = True
                    
                else:
                    show_speech_bubble(screen, 'Speechbubbles\Speechbubble_2_left.png', 'left', (5 * GRID_SIZE, 5 * GRID_SIZE),
                                        get_dialog('level3', 'dialog_10'))

            # If either player searches the fireplace
            if players['player1']['position'] == [4, 7] or players['player2']['position'] == [4, 7]:
                if level3_found_lighter:
                    cv_nav.update_player_positions(players['player1']['position'], players['player2']['position'])
                    use_lighter = cv_nav.get_use_lighter()

                    if (control_inputs.get('t', False) or use_lighter) and not level3_fire_started:
                        # Change map to level3_1 and set the speech bubble start time
                        screen, resources = map_init('maps\level_3\map_level3_1.png')
                        speech_bubble_start_time_fire = pygame.time.get_ticks()
                        speech_bubble_duration_fire = 10000  # 10 seconds
                        speech_bubble_active_fire = True

                        cv_nav.set_detection_classes(['hat', 'wooden spoon with shovel', 'blue lighter'])
                        
                        start_fire_sound.set_volume(2.0)
                        start_fire_sound.play()

                    
                        level3_fire_started = True
                        control_inputs['t'] = False  # Reset 't' after using it
                        use_lighter = False
                        
                    elif not level3_fire_started:
                        show_speech_bubble(screen, 'Speechbubbles\Speechbubble_2_left.png', 'left', (4 * GRID_SIZE, 7 * GRID_SIZE),
                                            get_dialog('level3', 'dialog_6'))
                else:
                    show_speech_bubble(screen, 'Speechbubbles\Speechbubble_2_left.png', 'left', (4 * GRID_SIZE, 7 * GRID_SIZE),
                                        get_dialog('level3', 'dialog_11'))

            # Display the speech bubble if active
            if 'speech_bubble_active_fire' in globals() and speech_bubble_active_fire:
                current_time = pygame.time.get_ticks()
                if current_time - speech_bubble_start_time_fire <= speech_bubble_duration_fire:
                    show_speech_bubble(
                        screen,
                        'Speechbubbles\Speechbubble_2_left.png',
                        'left',
                        (4 * GRID_SIZE, 7 * GRID_SIZE),
                        get_dialog('level3', 'dialog_7')
                    )
                else:
                    speech_bubble_active = False  # Deactivate the speech bubble after duration
                    print("Speech bubble duration ended.")
  
            # Handle Torch Pickup for Player 1
            if level3_fire_started and level3_knows_about_cave and not player1_has_torch:
                if players['player1']['position'] == [4, 7]:  # Replace [X, Y] with actual torch position
                    if not player1_has_stick:
                        # i should get a stick!
                        show_speech_bubble(
                            screen,
                            'Speechbubbles\Speechbubble_2_left.png',
                            'left',
                            (4 * GRID_SIZE, 7 * GRID_SIZE),
                            get_dialog('level3', 'dialog_8')
                        )

                    else:
                        cv_nav.update_player_positions(players['player1']['position'], players['player2']['position'])
                        use_stick = cv_nav.get_use_stick()
                        if control_inputs.get('z', False) or use_stick:
                            # Trigger speech bubble: "I grabbed a torch from the fire."
                            add_speech_bubble_event(screen, 'Speechbubbles\Speechbubble_2_left.png', (4 * GRID_SIZE, 7 * GRID_SIZE), get_dialog('level3', 'dialog_9'), 3000)
                            
                            use_torch_sound.set_volume(3.0)
                            use_torch_sound.play()
                            player1_has_torch = True

                            control_inputs['z'] = False  # Reset 'z' after using it
                            use_stick = False
                            print("Player 1 picked up a torch.")
            
            
            # Handle Torch Pickup for Player 2
        
            if level3_fire_started and level3_knows_about_cave and not player2_has_torch:
                if players['player2']['position'] == [4, 7]:  # Replace [X, Y] with actual torch position
                    if not player2_has_stick:
                        # i should get a stick!
                        show_speech_bubble(
                            screen,
                            'Speechbubbles\Speechbubble_2_left.png',
                            'left',
                            (4 * GRID_SIZE, 7 * GRID_SIZE),
                            get_dialog('level3', 'dialog_8')
                        )

                    else:
                        cv_nav.update_player_positions(players['player1']['position'], players['player2']['position'])
                       
                        use_stick = cv_nav.get_use_stick()
                        if control_inputs.get('z', False) or use_stick:
                            # Trigger speech bubble: "I grabbed a torch from the fire."
                            add_speech_bubble_event(screen, 'Speechbubbles\Speechbubble_2_left.png', (4 * GRID_SIZE, 7 * GRID_SIZE), get_dialog('level3', 'dialog_9'), 3000)
            
                            use_torch_sound.set_volume(3.0)
                            use_torch_sound.play()

                            player2_has_torch = True

                            print("Player 2 picked up a torch.")
                            use_stick = False
                            control_inputs['z'] = False  # Reset 'z' after using it
                            
                    




                # If both players have torches and are at the cave entrance
            
            
            ### winning condition for level 3 ###
            if player1_has_torch and player2_has_torch:
                if players['player1']['position'] == [14, 5] and players['player2']['position'] == [14, 5]:
                    visualize_mapchange = False
                    level = 4 
            # Draw players
            if not player2_has_torch:
                draw_player(players['player2']['position'], resources['player_2_image'], screen)
            else:
                draw_player(players['player2']['position'], resources['player_2_torch'], screen)
            
            if not player1_has_torch:
                draw_player(players['player1']['position'], resources['player_1_image'], screen)
            else:
                draw_player(players['player1']['position'], resources['player_1_torch'], screen)
                
            show_active_speech_bubbles(screen)
            pygame.display.flip()  # Update the screen once all elements are drawn
# ------------------- L E V E L 4 ------------------- #
        if level == 4:
            if not 'level4_initialized' in locals():
                # Initialize Level 4 variables
                adventurer_spoken = False
                visualize_mapchange = False
                cv_nav.set_detection_classes(['hat'])
                map_image = pygame.image.load('maps\level_4\map_level4_1_bright.png').convert_alpha()
                dark_map_image = pygame.image.load('maps\level_4\map_level4_1_dark.png').convert_alpha()
                players['player1']['position'] = [7, 3]  
                players['player2']['position'] = [7, 3]
                                            # Change the image of player 2 to player with torch:
       
                                            # Change the image of player 2 to player with torch:

                valid_path_level_4 = {(4,7),(13,7),(9,5), (6,3), (7, 3), (7,4), (7,5), (8,4), (8,4), (8,5), (8,6), (8,7), (8,8), (6,6), (7,6), (9,6), (10,6), (10,5), (9,5), (11,6), (11,5), (6,6), (5,6), (5,7), (6,7), (7,7), (8,7), (9,7), (10,7), (11,7), (12,7), (13,8), (12,8), (11,8), (10,8), (9,8), (8,8), (7,8), (6,8), (5,8), (4,8), (3,8), (2,8), (1,8), (1,7), (2,7), (3,7), (5,9), (6,9), (7,9), (8,9), (9,9), (10,9), (11,9), (12,9), (13,9), (13,8) }

                green_path_level_4 = {(11,5)}

                # green_path_level_4 = {(11,5), (5, 6), (13, 8), (6, 8), (11, 6)}

                level4_initialized = True

            screen.fill((255, 255, 255))
            screen.blit(resources['background'], (0, 0))

            if not visualize_mapchange:
                for i in range(255, 0, -1):
                    screen.fill((i, i, i))
                    pygame.display.flip()
                screen, resources = map_init('maps\level_4\map_level4_1_dark.png')  

                visualize_mapchange = True
                speech_bubble_start_time = pygame.time.get_ticks()
            update_player_position_from_cv_level1(players, control_inputs, screen.get_size(), valid_path_level_4)
            players_positions = [
                (players['player1']['position'][0], players['player1']['position'][1]),
                (players['player2']['position'][0], players['player2']['position'][1])
            ]

            ### Level 4: Trapped Adventurer Interaction ###
            if players['player1']['position'] == [11, 5] or players['player2']['position'] == [11, 5]:
                if not adventurer_spoken:
                    add_speech_bubble_event(screen, 'Speechbubbles\Speechbubble_1_left.png', (9 * GRID_SIZE, 5 * GRID_SIZE),
                                             get_dialog('level4', 'dialog_1'), 13000)
                    green_path_level_4 = {(6, 9), (1, 7)}
                    adventurer_spoken = True
            
            if adventurer_spoken:
                target_positions = {(6, 9), (1, 7)}
                current_positions = {(players['player1']['position'][0], players['player1']['position'][1]),
                                    (players['player2']['position'][0], players['player2']['position'][1])}
                
                cv_nav.set_checked_frame(5)
                
                if current_positions == target_positions:
                    
                    if 'timer_start_time' not in locals():
                        timer_start_time = pygame.time.get_ticks()
                        map_image = pygame.image.load('maps\level_4\map_level4_2_bright.png').convert_alpha()
                        dark_map_image = pygame.image.load('maps\level_4\map_level4_2_dark.png').convert_alpha()
                        add_speech_bubble_event(screen, 'Speechbubbles\Speechbubble_2_left.png', (8 * GRID_SIZE, 4 * GRID_SIZE),
                                                get_dialog('level4', 'dialog_2'), 3000)
                        green_path_level_4 = {(14, 7)}
                        # play stone_door_sound once:
                        stone_door_sound.set_volume(2.0)
                        stone_door_sound.play()
                        rune_magic_sound.play



            # Draw lighting effect and players
            draw_light_effect(screen, GRID_SIZE, players_positions, map_image, dark_map_image, valid_path_level_4, green_path_level_4)
            draw_player(players['player1']['position'], resources['player_1_torch'], screen)
            draw_player(players['player2']['position'], resources['player_2_torch'], screen)


            # Handling the countdown timer
            if 'timer_start_time' in locals():
                current_time = pygame.time.get_ticks()
                elapsed_time = (current_time - timer_start_time) // 1000  # Timer in seconds
                remaining_time = max(0, 17 - elapsed_time)
                #add (14,7) to valid_path_level_4:
                valid_path_level_4.add((14,7))
            

                if remaining_time > 0:
                    print("Time remaining:", remaining_time)
                    countdown_text = default_font.render(str(remaining_time), True, (255, 0, 0))
                    text_rect = countdown_text.get_rect(center=(15 * GRID_SIZE + GRID_SIZE // 2, 5 * GRID_SIZE + GRID_SIZE // 2))
                    screen.blit(countdown_text, text_rect)
                    
                    if players['player1']['position'] == [14, 7] and players['player2']['position'] == [14, 7]:
                        # Trigger win condition, go to next level
                        visualize_mapchange = False
                        level = 5
                else:
                    # Timer expired, revert maps and reset door state
                    map_image = pygame.image.load('maps\level_4\map_level4_1_bright.png').convert_alpha()
                    dark_map_image = pygame.image.load('maps\level_4\map_level4_1_dark.png').convert_alpha()
                    green_path_level_4 = {(6, 9), (1, 7)}
                    #drop (14,7) from valid_path_level_4:
                    valid_path_level_4.discard((14,7))
                    add_speech_bubble_event(screen, 'Speechbubbles\Speechbubble_1_left.png', (9 * GRID_SIZE, 5 * GRID_SIZE),get_dialog('level4', 'dialog_3'), 3000)
                    cv_nav.set_checked_frame(9)
                    stone_door_sound.play()
                    del timer_start_time
                        # Display speech bubbles and timer after the lighting effect
            show_active_speech_bubbles(screen)
            pygame.display.flip()
# ------------------- L E V E L 5 ------------------- #
        if level == 5:
            current_time_lvl5_1 = time.time()
            if not 'level5_initialized' in locals():
                # Initialization of level 5
                map_image = pygame.image.load('maps\level_5\map_level5_1_bright.png').convert_alpha()
                dark_map_image = pygame.image.load('maps\level_5\map_level5_1_dark.png').convert_alpha()
                cv_nav.set_checked_frame(9)
                cv_nav.set_detection_classes(['hat'])
                visualize_mapchange = False
                players['player1']['position'] = [8, 2]
                players['player2']['position'] = [8, 2]
                sequences = []
                rune_images = {
                    'blue': pygame.image.load(r'maps\level_5\rune_blue.png').convert_alpha(),
                    'green': pygame.image.load(r'maps\level_5\rune_green.png').convert_alpha(),
                    'red': pygame.image.load(r'maps\level_5\rune_red.png').convert_alpha()
                }
                rune_stone_images = {
                    'blue': pygame.image.load('maps\level_5\map_level5_0_blue.png').convert_alpha(),
                    'green': pygame.image.load('maps\level_5\map_level5_0_green.png').convert_alpha(),
                    'red': pygame.image.load('maps\level_5\map_level5_0_red.png').convert_alpha()
                }
                rune_stones_collected = {'blue': False, 'green': False, 'red': False}
                rune_positions = {'blue': [4, 6], 'green': [3, 8], 'red': [15, 6]}

                flaming_table = {
                    'blue': pygame.image.load(r'maps\level_5\rune_table_blue.png').convert_alpha(),
                    'red': pygame.image.load(r'maps\level_5\rune_table_red.png').convert_alpha(),
                    'green': pygame.image.load(r'maps\level_5\rune_table_green.png').convert_alpha()
                }

                regions = {
                    1: ((481, 346), (881, 591)),  # Top-left and bottom-right coordinates for Position 1
                    2: ((848, 346), (981, 500)),  # Top-left and bottom-right coordinates for Position 2
                    3: ((1244, 357), (1383, 621)) # Top-left and bottom-right coordinates for Position 3
                }
                recent_detections = {1: [], 2: [], 3: []}
                rebuild_sequence_dialog = False
                green_path_level_5 = {(4, 6), (3, 8), (15, 6)}
                cv_nav.checked_frame = 12
                valid_path_level_5 = {
                    # all 18 columns in row 0:
                    (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0),
                    # all 18 columns in row 1:
                    (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1),
                    # all 18 columns in row 2:
                    (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (15, 2), (16, 2), (17, 2), (18, 2),
                    # all 18 columns in row 3:
                    (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3), (7, 3), (8, 3), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3), (15, 3), (16, 3), (17, 3), (18, 3),
                    # all 18 columns in row 4:
                    (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4), (7, 4), (8, 4), (9, 4), (10, 4), (11, 4), (12, 4), (13, 4), (14, 4), (15, 4), (16, 4), (17, 4), (18, 4),
                    # all 18 columns in row 5:
                    (0, 5), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (9, 5), (10, 5), (11, 5), (12, 5), (13, 5), (14, 5), (15, 5), (16, 5), (17, 5), (18, 5),
                    # all 18 columns in row 6:
                    (0, 6), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6), (6, 6), (7, 6), (8, 6), (9, 6), (10, 6), (11, 6), (12, 6), (13, 6), (14, 6), (15, 6), (16, 6), (17, 6), (18, 6),
                    # all 18 columns in row 7:
                    (0, 7), (1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7), (8, 7), (9, 7), (10, 7), (11, 7), (12, 7), (13, 7), (14, 7), (15, 7), (16, 7), (17, 7), (18, 7),
                    # all 18 columns in row 8:
                    (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8), (9, 8), (10, 8), (11, 8), (12, 8), (13, 8), (14, 8), (15, 8), (16, 8), (17, 8), (18, 8),
                    # all 18 columns in row 9:
                    (0, 9), (1, 9), (2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9), (9, 9), (10, 9), (11, 9), (12, 9), (13, 9), (14, 9), (15, 9), (16, 9), (17, 9), (18, 9)
                }
                
                level5_2 = False
                level5_2_dialog = False
                rebuild_sequence = False
                sequence_1 = None
                sequence_2 = None
                sequence_3 = None
                check_interval_lvl5 = 10  
                level5_initialized = True

            screen.fill((255, 255, 255))
            screen.blit(resources['background'], (0, 0))

            if not visualize_mapchange:
                for i in range(255, 0, -1):
                    screen.fill((i, i, i))
                    pygame.display.flip()
                screen, resources = map_init('maps\level_5\map_level5_1_dark.png')
                visualize_mapchange = True
                speech_bubble_start_time = pygame.time.get_ticks()

            players_positions = [
                (players['player1']['position'][0], players['player1']['position'][1]),
                (players['player2']['position'][0], players['player2']['position'][1])
            ]
            update_player_position_from_cv_level1(players, control_inputs, screen.get_size(), valid_path_level_5)
            draw_light_effect(screen, GRID_SIZE, players_positions, map_image, dark_map_image, valid_path_level_5, green_path_level_5)
            
            
            # Picking up stones
            for color, position in rune_positions.items():
                if players['player1']['position'] == position or players['player2']['position'] == position:
                    if not rune_stones_collected[color]:
                        # Mark the stone as collected
                        rune_stones_collected[color] = True

                        # Trigger specific speech bubble based on the color of the stone found
                        speech_text = f"Look, \n I found a \n{color} magical stone\n!"
                        speech_position = (position[0] * GRID_SIZE, position[1] * GRID_SIZE)
                        add_speech_bubble_event(
                            screen,
                            'Speechbubbles\Speechbubble_2_left.png',
                            speech_position,
                            speech_text,
                            4000
                        )
                    
                        green_path_level_5.discard((position[0], position[1]))

            # Generate sequence:
            if all(rune_stones_collected.values()) and not rebuild_sequence and not level5_2:
                if sequence_1 == None:
                    # Generate a random sequence
                    pygame.time.wait(500)
                    sequence_1 = generate_sequence()
                    print("sequence_1:", sequence_1)  
                    # Visualize the sequence on the screen
                    visualize_sequence(screen, sequence_1, [(605, 469), (915, 420), (1318, 468)], dark_map_image, players, resources)
                if sequence_2 == None:
                    pygame.time.wait(500)
                    sequence_2 = generate_sequence()
                    print("sequence_2:", sequence_2)  
                    # Visualize the sequence on the screen
                    visualize_sequence(screen, sequence_2, [(605, 469), (915, 420), (1318, 468)], dark_map_image, players, resources)
                if sequence_3 == None:
                    pygame.time.wait(500)
                    sequence_3 = generate_sequence()
                    print("sequence_3:", sequence_3)  
                    # Visualize the sequence on the screen
                    visualize_sequence(screen, sequence_3, [(605, 469), (915, 420), (1318, 468)], dark_map_image, players, resources)
                    
                    rebuild_sequence = True
                   
                    green_path_level_5 = {(4, 7)}
                    sequences = [sequence_1, sequence_2, sequence_3]
      
            # rebuild sequences 
            if rebuild_sequence and not level5_2:
                recent_detections = {1: [], 2: [], 3: []}

                # Allow detection of magic stones by YOLO
                if 'book_class_activated' not in locals():  # Only activate once
                    cv_nav.set_detection_classes(['hat', 'frisbee'])
                    book_class_activated = True 

                detected_rune_positions = cv_nav.get_detected_objects()
                
                # Convert relative positions to absolute screen coordinates and draw them
                for detected in detected_rune_positions:
                    # Extract the color name by removing the 'rune_' prefix
                    rune_color = detected['label'].split('_')[1]  # Extracts 'blue', 'green', or 'red'
                    rel_pos = detected['relative_position']
                    abs_pos = (int(rel_pos[0] * screen.get_width()), int(rel_pos[1] * screen.get_height()))
                    
                    rune_image = rune_images.get(rune_color)
                    
                    if rune_image:
                        # Draw the rune image at the calculated position
                        rune_rect = rune_image.get_rect(center=abs_pos)
                        screen.blit(rune_image, rune_rect)

                    # Determine which region the rune falls into
                    assigned_region = None
                    for region_id, ((x1, y1), (x2, y2)) in regions.items():
                        if x1 <= abs_pos[0] <= x2 and y1 <= abs_pos[1] <= y2:
                            assigned_region = region_id
                            break

                    if assigned_region is not None:
                        # Add the detected color to the corresponding region's list
                        recent_detections[assigned_region].append(rune_color)

                        # Keep the list size manageable (e.g., store up to 5 recent detections)
                        if len(recent_detections[assigned_region]) > 5:
                            recent_detections[assigned_region].pop(0)


                # Calculate the remaining time for the blue flame
                remaining_time_lvl5 = max(0, check_interval_lvl5 - (current_time_lvl5_1 - last_check_time_lvl5))

                # Show the blue flame with a timer
                screen.blit(flaming_table['blue'], (0, 0))
                # Display the countdown timer on the flame
                timer_text = default_font.render(str(int(remaining_time_lvl5)), True, (255, 255, 255))
                screen.blit(timer_text, (960 - timer_text.get_width() // 2, 563 - timer_text.get_height() // 2))

                # Check if the 10-second interval has passed
                if current_time_lvl5_1 - last_check_time_lvl5 >= check_interval_lvl5:
                    # Update last check time
                    last_check_time_lvl5 = current_time_lvl5_1
                    
                    # Majority voting to determine the final color for each region
                    rebuild_sequence = []
                    for region_id in sorted(regions.keys()):
                        if recent_detections[region_id]:
                            most_common_color = max(set(recent_detections[region_id]), key=recent_detections[region_id].count)
                            rebuild_sequence.append(most_common_color)
                        else:
                            rebuild_sequence.append(None)

                    # Clear recent detections after processing
                    recent_detections = {1: [], 2: [], 3: []}

                    # Print the rebuild sequence
                    print("Rebuild sequence:", rebuild_sequence)

                    # Check if the rebuild sequence matches the current sequence
                    if rebuild_sequence == sequences[current_sequence_index_lvl5]:
                        print(f"Sequence {current_sequence_index_lvl5 + 1} matched successfully!")
                        # Show the green flame for 2 seconds
                        flame_display_time_lvl5 = current_time_lvl5_1 + 2
                        flame_color = 'green'
                        # Move to the next sequence
                        current_sequence_index_lvl5 += 1

                    else:
                        print(f"Rebuild sequence does not match Sequence {current_sequence_index_lvl5 + 1}.")
                        # Show the red flame for 2 seconds
                        flame_display_time_lvl5 = current_time_lvl5_1 + 2
                        flame_color = 'red'

                if control_inputs.get('i', False):
                    flame_display_time_lvl5 = current_time_lvl5_1 + 2
                    flame_color = 'green'
                    # Move to the next sequence
                    current_sequence_index_lvl5 = 2
                    control_inputs['i'] = False
                
                if control_inputs.get('o', False):
                    flame_display_time_lvl5 = current_time_lvl5_1 + 2
                    flame_color = 'green'
                    # Move to the next sequence
                    current_sequence_index_lvl5 = 3
                    control_inputs['o'] = False
            
                if current_sequence_index_lvl5 >= len(sequences):
                    # wait for 2 seconds before moving to the next level:
                    level5_2 = True 


                            # if any player is at position 5,5: rebuioild the sequence = False:
                   
                if players['player1']['position'] == [4,7] or players['player2']['position'] == [4, 7] and not rebuild_sequence_dialog:
                    add_speech_bubble_event(screen, 'Speechbubbles\Speechbubble_2_left.png', (4 * GRID_SIZE, 7 * GRID_SIZE), get_dialog('level5', 'dialog_3'), 5000)
                    rebuild_sequence_dialog = True
                
                if players['player1']['position'] == [4,7] and players['player2']['position'] == [4, 7]:
                    
                    # move players to the next position:
                    players['player1']['position'] = [12, 5]
                    players['player2']['position'] = [7, 5]
                    draw_player(players['player1']['position'], resources['player_1_torch'], screen)
                    draw_player(players['player2']['position'], resources['player_2_torch'], screen)
                    sequence_1 = None
                    sequence_2 = None
                    sequence_3 = None
                    sequences = []
                    rebuild_sequence = False
                    current_sequence_index_lvl5 = 0

            # Display the red or green flame if necessary
            if flame_display_time_lvl5 > current_time_lvl5_1:
                
                screen.blit(flaming_table[flame_color], (0, 0))

            # Overlay images for each rune if not yet collected AFTER draw_light_effect
            for color, position in rune_positions.items():
                if not rune_stones_collected[color]:
                    screen.blit(rune_stone_images[color], (position[0] * GRID_SIZE, position[1] * GRID_SIZE))

            if level5_2:
                # Display the green flame for 2 seconds
                screen.blit(flaming_table['green'], (0, 0))
                
                if not level5_2_dialog:
                    stone_door_sound.play()
                    cv_nav.set_detection_classes(['hat'])
                    map_image = pygame.image.load('maps\level_5\map_level5_2_bright.png').convert_alpha()
                    dark_map_image = pygame.image.load('maps\level_5\map_level5_2_dark.png').convert_alpha()
                    add_speech_bubble_event(screen, 'Speechbubbles\Speechbubble_2_left.png', (8 * GRID_SIZE, 4 * GRID_SIZE), get_dialog('level5', 'dialog_4'), 4000)
                    green_path_level_5 = {(4, 2)}
                    level5_2_dialog = True
                #if position is 1,4 of both players, level 6:
                if players['player1']['position'] == [4, 2] and players['player2']['position'] == [4, 2]:
                    
                    level = 6
                
            draw_player(players['player1']['position'], resources['player_1_torch'], screen)
            draw_player(players['player2']['position'], resources['player_2_torch'], screen)
            show_active_speech_bubbles(screen)
            pygame.display.flip()
# ------------------- L E V E L 6 ------------------- #
        if level == 6:
            if not 'level6_initialized' in locals():
                cv_nav.set_detection_classes(['hat', 'necklace'])
                map_image = pygame.image.load('maps\level_6\map_level6_1_bright.png').convert_alpha()
                dark_map_image = pygame.image.load('maps\level_6\map_level6_1_dark.png').convert_alpha()
                players['player1']['position'] = [8, 9]  
                players['player2']['position'] = [10, 9]
                visualize_mapchange = False
                start_level6 = False
                medallion_back = False
                pose_estimator_active = False
                frame_pose = None
                target_point = (955, 848)
                cv_nav.set_checked_frame(12)
                start_ritual_sound_playing = False
                ritualsteps = 1
                valid_path_level_6 = {(9, 7),(8,8), (9,8), (10,8), (8,9), (9,9), (10,9)}
                green_path_level_6 = {(9,7)}
                # Stature arms:
                start_ritual_channel = pygame.mixer.Channel(0)
                speech_bubble_display_time = None  # To track speech bubble duration
                ritual_max_energy = 20 
                # Define variables to track ritual steps and charge progress
                ritual_step_1_done = False
                ritual_overlay = pygame.image.load('maps\level_6\map_level6_2_circle.png').convert_alpha()
                ritual_overlay_alpha = 0   
                ritual_energy_charge = 0 
            
                ritual_step_2_done = False
                ritual_overlay_step2 = pygame.image.load('maps\level_6\map_level6_3_circle.png').convert_alpha()
                ritual_overlay_alpha_step2 = 0  
                ritual_energy_charge_step2 = 0
                level6_1 = False
                level6_initialized = True

            screen.fill((255, 255, 255))
            screen.blit(resources['background'], (0, 0))
            update_player_position_from_cv_level1(players, control_inputs, screen.get_size(), valid_path_level_6)

            if not visualize_mapchange:
                for i in range(255, 0, -1):
                    screen.fill((i, i, i))
                    pygame.display.flip()
                screen, resources = map_init('maps\level_6\map_level6_1_bright.png')  

                visualize_mapchange = True
                speech_bubble_start_time = pygame.time.get_ticks()

            players_positions = [
                (players['player1']['position'][0], players['player1']['position'][1]),
                (players['player2']['position'][0], players['player2']['position'][1]) ]

            ### auto speech bubble starts here ###  get_dialog('level4', 'dialog_1')
            if not start_level6:
                add_speech_bubble_event(screen, 'Speechbubbles\Speechbubble_1_left.png', (9 * GRID_SIZE, 5 * GRID_SIZE),
                                              get_dialog('level6', 'dialog_1'), 5000)
                start_level6 = True
            

            if (players['player1']['position'] == [9, 7] or players['player2']['position'] == [9, 7]):
                cv_nav.update_player_positions(players['player1']['position'], players['player2']['position'])
                medallion_back = cv_nav.get_medallion_back()
                        
                if control_inputs.get('t', False) or medallion_back:
                    add_speech_bubble_event(screen, 'Speechbubbles\Speechbubble_1_left.png', (9 * GRID_SIZE, 5 * GRID_SIZE), get_dialog('level6', 'dialog_2'), 8000)

                    # delete the green path:
                    green_path_level_6.discard((11, 5))
                    players['player1']['position'] = [6, 8]  
                    players['player2']['position'] = [12, 8]

                    valid_path_level_6 = {()}
                    green_path_level_6 = {()}
                    medallion_back = True

            # Initialize PoseEstimationRitual when medallion is back
            if medallion_back and not pose_estimator_active:
                pose_estimator = PoseEstimationRitual()  # Initialize YOLO11n-pose
                pose_estimator_active = True
            else:
                draw_grid_overlay(screen, GRID_SIZE, valid_path_level_6, green_path=green_path_level_6, red_path=None)

            if pose_estimator:
                # Process the frame and get both the annotated webcam frame and the Pygame surface
                annotated_webcam_frame, pygame_surface = pose_estimator.process_frame(frame, screen)

                # Display the annotated OpenCV frame for the live webcam feed
                cv2.imshow('Webcam Feed', annotated_webcam_frame)

                # Render the Pygame surface on the screen using blit
                screen.blit(pygame_surface, (0, 0))

            ### Ritual Pose Detection: 
                # Step 1: Close the circle for 10 consecutive frames
                if ritualsteps == 1:

                    if pose_estimator.is_circle_closed():
                        
                        ritual_energy_charge += 1
                        ritual_overlay_alpha = min(255, (ritual_energy_charge / ritual_max_energy) * 255)
                        
                        #play start_ritual_sound for 0.5 s:
                        if start_ritual_channel is None or not start_ritual_channel.get_busy():
                            start_ritual_channel = pygame.mixer.find_channel()
                            if start_ritual_channel is not None:
                                start_ritual_channel.play(start_ritual_sound)
                        
                    else:
                        # Reset energy charge if the circle is not closed
                        ritual_energy_charge -= 1
                        ritual_energy_charge = max(0, ritual_energy_charge)
                        ritual_overlay_alpha = min(255, (ritual_energy_charge / ritual_max_energy) * 255)
                        if start_ritual_channel is not None:
                            start_ritual_channel.stop()
                            start_ritual_channel = None  # Reset the channel

                    # If the circle is closed for 10 frames, mark step 1 as done
                    if ritual_energy_charge >= ritual_max_energy:
                        start_ritual_channel.stop()
                        
                        ritual_step_1_done = True
                        ritual_energy_charge = 0
                        speech_bubble_display_time = pygame.time.get_ticks()  # Show speech bubble for feedback
                        ritualsteps = 2
                        pose_estimator.update_ritualsteps(ritualsteps) 

                    ritual_overlay.set_alpha(ritual_overlay_alpha)
                    screen.blit(ritual_overlay, (0, 0))
                    # Display a speech bubble if step 1 is completed (show for 2 seconds)
                    if ritual_step_1_done and (pygame.time.get_ticks() - speech_bubble_display_time < 4000):
                        add_speech_bubble_event(screen, 'Speechbubbles\Speechbubble_1_left.png', (9 * GRID_SIZE, 5 * GRID_SIZE),
                                                get_dialog('level6', 'dialog_3'), 4000)
                # Step 2: Check if all arms are held up and charge energy
                if ritualsteps == 2:
                    
                    print("Checking arms-up: ", pose_estimator.check_arms_held_up())
                    print("Ritual Energy Charge Step2: ", ritual_energy_charge_step2)
                    print("Ritual Overlay Alpha Step2: ", ritual_overlay_alpha_step2)


                    all_arms_up = pose_estimator.check_arms_held_up()  # Check if all arms are up
                    

                    if all_arms_up:
                        ritual_energy_charge_step2 += 1
                        ritual_overlay_alpha_step2 = min(255, (ritual_energy_charge_step2 / ritual_max_energy) * 255)
                        print("ritual_energy_charge_step2: ", ritual_energy_charge_step2)
                    else:
                        ritual_energy_charge_step2 -= 1
                        ritual_energy_charge_step2 = max(0, ritual_energy_charge_step2)
                        ritual_overlay_alpha_step2 = min(255, (ritual_energy_charge_step2 / ritual_max_energy) * 255)

                    # If the arms are held up for enough frames, transition to the next ritual step
                    if ritual_energy_charge_step2 >= ritual_max_energy:
                        teleport_sound.play()
                        ritual_step_2_done = True
                        speech_bubble_display_time = pygame.time.get_ticks()  # Show speech bubble for feedback
                        # You can proceed to the next step or set additional logic for the ritual here

                    # Adjust the alpha of the glowing lines based on energy charge
                    ritual_overlay_step2.set_alpha(ritual_overlay_alpha_step2)
                    screen.blit(ritual_overlay_step2, (0, 0))

                    # Optional: Display a speech bubble if step 2 is completed (show for 4 seconds)
                    if ritual_step_2_done and (pygame.time.get_ticks() - speech_bubble_display_time < 4000):
                        add_speech_bubble_event(screen, 'Speechbubbles\Speechbubble_2_left.png', (9 * GRID_SIZE, 5 * GRID_SIZE),
                                                get_dialog('level6', 'dialog_4'), 4000)
                        #wait and then teleport:
                        pygame.time.wait(4000)

                        #blend over once to a white screen:
                        fade_white_surface = pygame.Surface(screen.get_size())
                        fade_white_surface.fill((255, 255, 255))
                        for alpha in range(0, 256):
                            fade_white_surface.set_alpha(alpha)
                            screen.blit(fade_white_surface, (0, 0))
                            pygame.display.flip()
                        if alpha == 255: 
                            level = 7

                        
                    
            draw_player(players['player1']['position'], resources['player_1_torch'], screen)
            draw_player(players['player2']['position'], resources['player_2_torch'], screen)

            show_active_speech_bubbles(screen)
            pygame.display.flip()
# ------------------- L E V E L 7 ------------------- #
        if level == 7:
            # Initialize the outro sequence if it's not done yet
            if not 'level7_initialized' in locals():
                adventurer = pygame.image.load('maps\level_7\map_level3_4_adventurer.png').convert_alpha()
                teleport_beam = pygame.image.load('maps\level_7\map_level3_3.png').convert_alpha()
                start_level7 = False
                visualize_mapchange = False
                teleport_beam_alpha = 0
                end_ritual_sound.play()
                
                speech_bubble_shown = False  # Track speech bubble display
                speech_bubble_duration = 8000  # Time in milliseconds
                fadeout_beam = False  # Track teleport beam fade-out
                teleport_beam_alpha_fadeout = 255  # Initial alpha for fade-out
                speech_bubble_end_time = 0
                level7_initialized = True
            
            screen.fill((255, 255, 255))
            screen.blit(resources['background'], (0, 0))
            
            # Visualization of Map Level Change
            if not visualize_mapchange:
                for i in range(255, 0, -1):
                    screen.fill((i, i, i))
                    pygame.display.flip()
                screen, resources = map_init('maps\level_7\map_level3.png')  # Load map level 3
                players['player1']['position'] = [9, 9]
                players['player2']['position'] = [9, 8]
                visualize_mapchange = True
                pygame.time.wait(500)
                speech_bubble_start_time = pygame.time.get_ticks()
                start_level7 = True

            # Handle teleport beam and speech bubble
            if start_level7:
                   
                if teleport_beam_alpha < 255:
                    teleport_beam_alpha += 5  # Increase alpha for fade-in effect
                else:
                    teleport_beam_alpha = 255  # Max alpha

                teleport_beam.set_alpha(teleport_beam_alpha)
                
                # Once beam is fully visible, show players and adventurer
                if teleport_beam_alpha == 255:
                    draw_player(players['player1']['position'], resources['player_1_image'], screen)
                    draw_player(players['player2']['position'], resources['player_2_image'], screen)
                    screen.blit(adventurer, (0,0))
                    fadeout_beam = True

                # Start fade-out of the teleport beam after it's fully shown
                if fadeout_beam:
                    add_speech_bubble_event(screen, 'Speechbubbles\Speechbubble_1_left.png', (10 * GRID_SIZE, 7 * GRID_SIZE),
                                            get_dialog('level7', 'dialog_1'), speech_bubble_duration)# Begin beam fade-out
                    if teleport_beam_alpha_fadeout > 0:
                        teleport_beam_alpha_fadeout -= 10  # Decrease alpha for fade-out effect
                        teleport_beam.set_alpha(teleport_beam_alpha_fadeout)
                    else:                        
                        teleport_beam.set_alpha(teleport_beam_alpha)
                        #delay ?
                        speech_bubble_shown = True
                        fadeout_beam = False
                        

                # Fade the screen to black after showing the outro sequence
            if speech_bubble_shown:
                end_screen = pygame.image.load(r'maps\level_7\7_EndScreen.png').convert_alpha()  # Load the end screen image
                alpha_outro = 0  # Start with transparent end screen

                # Smoothly blend to the end screen using the game loop
                while alpha_outro <= 255:
                    end_screen.set_alpha(alpha_outro)  # Apply the increasing alpha to the end screen

                    # Blit the end screen with increasing alpha for the fade-in effect
                    screen.blit(end_screen, (0, 0))
                    pygame.display.flip()

                    # Increment the alpha for the fade effect
                    alpha_outro += 2  # Adjust this value for a smoother or faster fade
                    clock.tick(10)  # Control the speed of the fade, adjust FPS as needed

                # After the fade-in is complete, show the end screen for 5 seconds
                screen.blit(end_screen, (0, 0))  # Ensure the full end screen is visible
                pygame.display.flip()

                # Wait for 5 seconds after the end screen is fully shown
                pygame.time.wait(5000)

                # End the game
                running = False  # Set running to False to exit the game loop
            else:
                screen.blit(teleport_beam, (0, 0))
                show_active_speech_bubbles(screen)
                pygame.display.flip()


# ------------------- S h ow  ------------------- #
        # Process frame and display during running the game loop 
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame")
            continue
        frame = cv2.flip(frame, 1)
        if not pose_estimator and level != 7:
            frame = cv_nav.process_frame(frame,players['player1']['position'],players['player2']['position'])
            cv2.imshow('Webcam Feed', frame)
        # else:  
        #     cv2.imshow('Webcam Feed', frame)

        print("Player 2: ", players['player2']['position'])
        clock.tick(FPS)


        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    exit_game()

if __name__ == "__main__":
    game_loop()

    


Ultralytics 8.3.217  Python-3.9.19 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12281MiB)
Setup complete  (24 CPUs, 31.9 GB RAM, 808.6/931.0 GB disk)


c:\Users\EB\.conda\envs\yolov8_3\lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


[CVN] set_classes() done once at init on device: cuda
[CVN] detection whitelist -> ['hat']
[CVN] predict -> 1 result(s)
[CVN] One hat: brightness 137.4 -> white hat
[CVN] Draw hat label 'Player 2 0.97' at (706,460)
Player 2:  [7, 9]
[CVN] predict -> 1 result(s)
[CVN] One hat: brightness 144.5 -> white hat
[CVN] Draw hat label 'Player 2 0.97' at (708,461)
Player 2:  [7, 9]
[CVN] predict -> 1 result(s)
[CVN] One hat: brightness 147.2 -> white hat
[CVN] Draw hat label 'Player 2 0.97' at (709,460)
Player 2:  [7, 9]
[CVN] predict -> 1 result(s)
[CVN] One hat: brightness 174.8 -> white hat
[CVN] Draw hat label 'Player 2 0.97' at (707,460)
Player 2:  [7, 9]
[CVN] predict -> 1 result(s)
[CVN] One hat: brightness 165.2 -> white hat
[CVN] Draw hat label 'Player 2 0.96' at (707,461)
Player 2:  [7, 9]
[CVN] predict -> 1 result(s)
[CVN] One hat: brightness 162.3 -> white hat
[CVN] Draw hat label 'Player 2 0.97' at (707,461)
Player 2:  [7, 9]
[CVN] predict -> 1 result(s)
[CVN] One hat: brightness 16

SystemExit: 

c:\Users\EB\.conda\envs\yolov8_3\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
